In [1]:
import pandas as pd
import numpy as np

import torch

import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, LearningRateMonitor
from pytorch_lightning.loggers import TensorBoardLogger

from pytorch_forecasting import Baseline, TemporalFusionTransformer, TimeSeriesDataSet
from pytorch_forecasting.data import GroupNormalizer
from pytorch_forecasting.metrics import SMAPE, PoissonLoss, QuantileLoss
from pytorch_forecasting.models.temporal_fusion_transformer.tuning import optimize_hyperparameters

In [2]:
data_path = "./Refined_Data/Grouped_Data/Input_Data2.csv"

building_num = [[34]]

repeat = 10

In [3]:
data = pd.read_csv(data_path, parse_dates = ["date_time"])

data['num']     =   data['num'].apply(str)
data['day_of_Week']     =   data['day_of_Week'].apply(str)
data['day_of_month']    =   data['day_of_month'].apply(str)
data['24Hour']  =   data['24Hour'].apply(str)
data['holiday'] =   data['holiday'].apply(str)
data['Weekend'] =   data['Weekend'].apply(str)
data['energy_group'] = data['energy_group'].apply(str)
data['hour_cat']=   data['hour_cat'].apply(str)

In [4]:
data_bag = [data.loc[data["num"] == str(building_num[i])].copy() for i in range(len(building_num))]

data_bag = []
for buildings in building_num:
    temp = data.loc[data["num"] == str(buildings[0])].copy()
    for num_idx in range(1, len(buildings)):
        temp = pd.concat([temp, data.loc[data["num"] == str(buildings[num_idx])].copy()])
    data_bag.append(temp)

In [5]:
for ___ in range(repeat):
    for idx in range(len(data_bag)):

        data = data_bag[idx]

        torch.cuda.empty_cache()

        max_prediction_length = 168
        max_encoder_length = 336
        training_cutoff = data["time_idx"].max() - max_prediction_length

        training = TimeSeriesDataSet(
            data[lambda x: x.time_idx <= training_cutoff],
            time_idx="time_idx",
            target="kWH",
            group_ids=["num"],
            min_encoder_length=max_encoder_length,
            max_encoder_length=max_encoder_length,
            min_prediction_length=max_prediction_length,
            max_prediction_length=max_prediction_length,
            static_categoricals=["num", "energy_group"],
            static_reals=["non_electric_aircondition", "sunlight"],
            time_varying_known_categoricals=["day_of_Week", "day_of_month", "24Hour", "holiday", "Weekend", "hour_cat"],
            time_varying_known_reals=["C", "m/s", "wet", "mm", "hr", "time_idx", "day", "Week", "perceived_temperature", "discomfort_index"],
            time_varying_unknown_categoricals=[],
            time_varying_unknown_reals=["kWH"],
            add_relative_time_idx=True,
            add_target_scales=True,
            add_encoder_length=True
        )

        # create validation set (predict=True) which means to predict the last max_prediction_length points in time
        # for each series
        validation = TimeSeriesDataSet.from_dataset(training, data, predict=True, stop_randomization=True)

        # create dataloaders for model
        batch_size = 64  # set this between 32 to 128
        train_dataloader = training.to_dataloader(train=True, batch_size=batch_size, num_workers=0)
        val_dataloader = validation.to_dataloader(train=False, batch_size=batch_size * 10, num_workers=0)

        # configure network and trainer
        early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=1e-4, patience=20, verbose=False, mode="min")
        lr_logger = LearningRateMonitor()  # log the learning rate
        logger = TensorBoardLogger("lightning_logs")  # logging results to a tensorboard

        trainer = pl.Trainer(
            max_epochs=500,
            gpus=1,
            weights_summary="top",
            gradient_clip_val=0.14,
            limit_train_batches=10, 
            callbacks=[lr_logger, early_stop_callback],
            logger=logger,
        )


        tft = TemporalFusionTransformer.from_dataset(
            training,
            learning_rate=0.03,
            hidden_size=64,
            lstm_layers = 2,
            attention_head_size=4,
            dropout=0.15,
            hidden_continuous_size=8,
            output_size=1,
            loss=SMAPE(),
            log_interval=0,
            reduce_on_plateau_patience=4,
        )

        # fit network
        trainer.fit(
            tft,
            train_dataloader=train_dataloader,
            val_dataloaders=val_dataloader,
        )

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name                               | Type                            | Params
----------------------------------------------------------------------------------------
0  | loss                               | SMAPE                           | 0     
1  | logging_metrics                    | ModuleList                      | 0     
2  | input_embeddings                   | MultiEmbedding                  | 618   
3  | prescalers                         | ModuleDict                      | 272   
4  | static_variable_selection          | VariableSelectionNetwork        | 9.0 K 
5  | encoder_variable_selection         | VariableSelectionNetwork        | 25.3 K
6  | decoder_variable_selection         | VariableSelectionNetwork        | 23.2 K
7  | static_context_variable_selection  | GatedResidualNetwork            | 16.8 K
8  | static_context_initial_hidden_lstm | GatedR

Validation sanity check:   0%|          | 0/1 [00:00<?, ?it/s]

C:\Users\Eunchan\anaconda3\envs\torch1\lib\site-packages\pytorch_lightning\utilities\distributed.py:69: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


Epoch 0:   0%|          | 0/11 [00:00<?, ?it/s] 

C:\Users\Eunchan\anaconda3\envs\torch1\lib\site-packages\pytorch_lightning\utilities\distributed.py:69: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


...

Validation sanity check:   0%|          | 0/1 [00:00<?, ?it/s]

C:\Users\Eunchan\anaconda3\envs\torch1\lib\site-packages\pytorch_lightning\utilities\distributed.py:69: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


Epoch 0:   0%|          | 0/11 [00:00<?, ?it/s] 

C:\Users\Eunchan\anaconda3\envs\torch1\lib\site-packages\pytorch_lightning\utilities\distributed.py:69: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


Epoch 31: 100%|██████████| 11/11 [00:06<00:00,  1.67it/s, loss=0.115, v_num=9, val_loss=0.250, train_loss_step=0.113, train_loss_epoch=0.114]
